# AskQL Demo

This notebook demonstrates the AskQL natural language to SQL agent.

**Features:**
- Convert natural language questions to SQL
- Execute queries on DuckDB
- Automatic error correction
- Query validation and safety checks

## Setup

In [2]:
from askQL import BasicSQLAgent
import os
import pandas as pd
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize agent
agent = BasicSQLAgent(os.getenv("OPENAI_API_KEY"))
print("Agent initialized successfully!")

Schema loaded from CSV
Agent initialized successfully!


## Database Schema

Our sample database has 3 tables:
- **customers** (50 rows): customer_id, name, email, age, city, signup_date
- **products** (30 rows): product_id, product_name, category, price, in_stock
- **orders** (200 rows): order_id, customer_id, product_id, quantity, order_date, total_amount

## Example 1: Simple Count Query

In [3]:
result = agent.query("How many customers are there?")

print("Generated SQL:")
print(result['sql'])
print("\nResults:")
print(result['results'])


Processing: How many customers are there?
Generated SQL: SELECT COUNT(customer_id) AS total_customers
FROM customers;
SQL validation passed
Executing query...
Query executed successfully (1 rows in 0.00s)
Results formatted successfully
Generated SQL:
SELECT COUNT(customer_id) AS total_customers
FROM customers;

Results:
 total_customers
              50

(1 rows returned)


## Example 2: Filtering and Sorting

In [ ]:
result = agent.query("Show me the top 5 most expensive products")

print("Generated SQL:")
print(result['sql'])
print("\nResults:")
print(result['results'])
print(f"\nExecution time: {result['execution_time']:.4f}s")

## Example 3: Aggregation with Joins

In [ ]:
result = agent.query("Show customer names with their total spending")

print("Generated SQL:")
print(result['sql'])
print("\nResults (first 5):")
print(result['results'])
print(f"\nTotal customers: {result['rows']}")

## Example 4: Access All Raw Results

In [ ]:
result = agent.query("Show customer names with their total spending")

# Convert raw results to DataFrame
df = pd.DataFrame(result['raw_results'])

print(f"Total rows: {len(df)}")
print("\nAll results:")
df

## Example 5: Complex Query - Top Customers

In [ ]:
result = agent.query(
    "Show me the top 10 customers by total spending, "
    "including their name, city, total spent, and number of orders"
)

print("Generated SQL:")
print(result['sql'])
print("\nResults:")
print(result['results'])

## Example 6: Date Filtering

In [ ]:
result = agent.query("How many orders were placed in 2024?")

print("Generated SQL:")
print(result['sql'])
print("\nResults:")
print(result['results'])

## Example 7: Category Analysis

In [ ]:
result = agent.query("Show total revenue by product category")

print("Generated SQL:")
print(result['sql'])
print("\nResults:")
print(result['results'])

## Example 8: Visualization

In [ ]:
import matplotlib.pyplot as plt

result = agent.query("Show total revenue by product category")
df = pd.DataFrame(result['raw_results'])

# Create bar chart
plt.figure(figsize=(10, 6))
plt.bar(df.iloc[:, 0], df.iloc[:, 1])
plt.xlabel('Category')
plt.ylabel('Total Revenue')
plt.title('Revenue by Product Category')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"Generated SQL: {result['sql']}")

## Example 9: Validation Test (Should Fail)

In [ ]:
# This should fail validation (not a SELECT query)
result = agent.query("Delete all customers")

print(f"Success: {result['success']}")
print(f"Generated SQL: {result['sql']}")
print(f"\nValidation Error: {result['validation_error']}")

## Example 10: Multiple Queries at Once

In [ ]:
questions = [
    "What is the average order value?",
    "How many products are currently in stock?",
    "Which city has the most customers?",
    "What is the total revenue from Electronics category?"
]

results_summary = []

for question in questions:
    result = agent.query(question)
    results_summary.append({
        'Question': question,
        'Success': result['success'],
        'SQL': result['sql'][:50] + '...' if len(result['sql']) > 50 else result['sql'],
        'Rows': result['rows']
    })

pd.DataFrame(results_summary)

## Example 11: Export Results to CSV

In [ ]:
result = agent.query("Show all customers with their total spending, sorted by spending")

# Convert to DataFrame and save
df = pd.DataFrame(result['raw_results'])
df.to_csv('customer_spending.csv', index=False)

print(f"Exported {len(df)} rows to customer_spending.csv")
print("\nFirst few rows:")
print(result['results'])

## Example 12: Check Query Statistics

In [ ]:
result = agent.query("Show customer names with their total spending")

print("Query Statistics:")
print(f"  Success: {result['success']}")
print(f"  Rows returned: {result['rows']}")
print(f"  Execution time: {result['execution_time']:.4f}s")
print(f"  Retry count: {result['retry_count']}")
print(f"  SQL length: {len(result['sql'])} characters")
print(f"\nGenerated SQL:")
print(result['sql'])

## Try Your Own Query

In [ ]:
# Enter your question here
my_question = "Show me products that cost more than 1000"

result = agent.query(my_question)

print("Generated SQL:")
print(result['sql'])
print("\nResults:")
print(result['results'])

# Show raw data as DataFrame
if result['success']:
    df = pd.DataFrame(result['raw_results'])
    display(df)

## Summary

This notebook demonstrated:
- Simple and complex SQL queries from natural language
- Accessing raw results vs formatted output
- Data visualization
- Validation and error handling
- Exporting results to CSV
- Query statistics and metadata

**Key features:**
- Displays first 5 rows by default
- All data available in `result['raw_results']`
- Automatic error correction (up to 3 retries)
- SELECT-only validation for safety
- Real query execution on DuckDB